In [1]:
!pip install deepface opencv-python tf-keras

In [2]:
import os
import cv2
import time
import pandas as pd
from datetime import datetime
from deepface import DeepFace


In [3]:
from deepface import DeepFace
import os

def identify_person(img_path, db_path, model_name="Facenet512", threshold=0.3):
    try:
        # Perform face recognition
        df = DeepFace.find(
            img_path=img_path,
            db_path=db_path,
            model_name=model_name,
            enforce_detection=False,
            silent=True
        )[0]  # the returned result is a list of DataFrames

        if df.shape[0] == 0:
            return "Unknown"

        # Check if any match is below the threshold
        best_match = df.iloc[0]
        if best_match["distance"] <= threshold:
            # Extract person's name from path
            identity_path = best_match["identity"]
            person_name = os.path.basename(os.path.dirname(identity_path))
            return person_name
        else:
            return "Unknown"

    except Exception as e:
        print("Detection error:", e)
        return "Unknown"


In [4]:
def mark_attendance(name):
    now = datetime.now()
    dt_string = now.strftime('%Y-%m-%d %H:%M:%S')
    df = pd.DataFrame([[name, dt_string]], columns=["Name", "Time"])
    df.to_csv("attendance.csv", mode='a', header=not os.path.exists("attendance.csv"), index=False)
    print(f"[LOGGED] {name} at {dt_string}")


In [5]:
attendance_logged = set()
db_path = "faces_db"
cap = cv2.VideoCapture(0)
start_time = time.time()
max_duration = 2 * 60  # 2 minutes

while True:
    ret, frame = cap.read()
    if not ret:
        break

    name = identify_person(frame, db_path, threshold=0.3, model_name='Facenet512')

    if name != "Unknown":
        if name not in attendance_logged:
            mark_attendance(name)
            attendance_logged.add(name)

    # Draw name on frame
    cv2.putText(frame, name, (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1,
                (0, 255, 0) if name != "Unknown" else (0, 0, 255), 2)

    cv2.imshow("Face Attendance", frame)

    # Stop if 2 minutes pass or 'q' is pressed
    if time.time() - start_time > max_duration:
        print("[INFO] 2 minutes passed. Exiting...")
        break
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


[LOGGED] akshay at 2025-04-22 01:25:12
[LOGGED] pratik at 2025-04-22 01:25:23
[LOGGED] Dhruv at 2025-04-22 01:25:30
[LOGGED] hariom at 2025-04-22 01:25:55
[LOGGED] pritesh at 2025-04-22 01:26:05
[INFO] 2 minutes passed. Exiting...
